### Libraries

In [23]:
import math
import numpy as np

# import pandas 
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Constants

In [24]:
# @title Gauss-Laguerre constants
# Notation: (A_i, x_i)

GAUSS_LAGUERRE = [
    (0.458964, 0.222847),
    (0.417, 1.188932),
    (0.113373, 2.992736),
    (0.0103992, 5.775144),
    (0.000261017, 9.837467),
    (0.000000898548, 15.982874)
]

### PDF, CDF of Weibull distribution

In [25]:
# @title Weibull distribution function
# Uses Gauss-Laguerre quadratures
def f_Weibull (t, k, mu):
    denomArea = 0
    
    for node in GAUSS_LAGUERRE:
        denomArea += node[0] * (node[1]**(1/k))

    lbda = mu / denomArea

    return (k/lbda) * ((t/lbda)**(k-1)) * (math.e ** (-(t/lbda)**k))

In [26]:
# @title Integral of Weibull function
# Using Gauss-Laguerre quadratures
def cdf (t, k, mu):
    area = 0
    
    for node in GAUSS_LAGUERRE:
        area += node[0] * (math.e**node[1]) * f_Weibull(t + node[1], k, mu)

    return 1 - area

In [27]:
# @title CDF values
t_values, y_CDF_05, y_CDF_1, y_CDF_2 = [], [], [], []

for t in range(0, 251, 1):
    t /= 100
    t_values.append(t)
    y_CDF_05.append(cdf(t, 0.5, 2))
    y_CDF_1.append(cdf(t, 1, 1))
    y_CDF_2.append(cdf(t, 2, math.sqrt(math.pi)/2))

In [28]:
# @title CDF plotting
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=t_values, y=y_CDF_05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
    go.Scatter(x=t_values, y=y_CDF_1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
    go.Scatter(x=t_values, y=y_CDF_2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
])
fig.update_layout(
    height=1080*0.5,
    width=1920*0.6,
    xaxis_title="t",
    yaxis_title="alpha"
)
fig.show()


### Root-finding problem

In [29]:
# @title Inverse survivability
# solved as root-finding problem using bisection method
def surv_inv (alpha, a, b, k, mu):
    isDiffSign = lambda x1, x2 : np.sign(x1) != np.sign(x2)

    f = lambda t : cdf(t, k, mu) - alpha

    tol = 1.0e-9

    fa = f(a)
    fb = f(b)

    if fa == 0.0:
        return a
    
    if fb == 0.0:
        return b
    
    print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
    print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))
    
    if not isDiffSign(fa, fb):
        print("---> no root at: ", alpha)
        return None
    
    n = int (math.ceil (math.log(abs(b-a)/tol) / math.log(2.0)))

    print("---> iterations: ", n)

    for i in range(n):
        print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
        print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))

        c = 0.5 * (a + b)
        fc = f(c)

        if fc == 0.0:
            print("---> returning c =", c)
            return c

        if isDiffSign(fa, fc):
            b = c
            fb = fc
        
        elif isDiffSign(fb, fc):
            a = c
            fa = fc

    return 0.5 * (a+b)

#### Evaluation

In [30]:
# List for values storing
x_values = []
y_t_05 = []
y_t_1 = []
y_t_2 = []

# Average life expectancy
mu = 78

In [31]:
# For alpha in (0,1) with 0.01 step
for alpha in range(1, 100, 1):
    alpha /= 100

    x_values.append(alpha)

    # y_t_05.append(surv_inv(alpha, 0, 250, 0.5, mu))
    # y_t_1.append(surv_inv(alpha, 0, 250, 1, mu))
    y_t_2.append(surv_inv(alpha, 0, 250, 2, mu))

-> CDF(0) - 0.01 = 0.9384380078536639
-> CDF(250) - 0.01 = 0.9897920701642423
---> no root at:  0.01
-> CDF(0) - 0.02 = 0.9284380078536639
-> CDF(250) - 0.02 = 0.9797920701642423
---> no root at:  0.02
-> CDF(0) - 0.03 = 0.9184380078536639
-> CDF(250) - 0.03 = 0.9697920701642423
---> no root at:  0.03
-> CDF(0) - 0.04 = 0.9084380078536639
-> CDF(250) - 0.04 = 0.9597920701642423
---> no root at:  0.04
-> CDF(0) - 0.05 = 0.8984380078536639
-> CDF(250) - 0.05 = 0.9497920701642423
---> no root at:  0.05
-> CDF(0) - 0.06 = 0.888438007853664
-> CDF(250) - 0.06 = 0.9397920701642424
---> no root at:  0.06
-> CDF(0) - 0.07 = 0.878438007853664
-> CDF(250) - 0.07 = 0.9297920701642424
---> no root at:  0.07
-> CDF(0) - 0.08 = 0.868438007853664
-> CDF(250) - 0.08 = 0.9197920701642424
---> no root at:  0.08
-> CDF(0) - 0.09 = 0.858438007853664
-> CDF(250) - 0.09 = 0.9097920701642424
---> no root at:  0.09
-> CDF(0) - 0.1 = 0.848438007853664
-> CDF(250) - 0.1 = 0.8997920701642423
---> no root at:  0.

#### Plotting

In [32]:
# @title Inverse survivability plot
fig = go.Figure()
fig.add_traces([
    # go.Scatter(x=x_values, y=y_t_05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
    # go.Scatter(x=x_values, y=y_t_1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
    go.Scatter(x=x_values, y=y_t_2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
])
fig.update_layout(
    height=1080*0.5,
    width=1920*0.5,
    xaxis_title="alpha",
    yaxis_title="t"
)
fig.show()

In [33]:
print(x_values)
print(y_t_05)
print(y_t_1)
print(y_t_2)

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
[]
[]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,